# PaddlePaddle实现的端到端自动语音识别

**作者:** [夜雨飘零](https://github.com/yeyupiaoling)<br/>
**日期:** 2021.05<br/>
**摘要:** 本示例教程演示如何在PaddlePaddle实现的端到端自动语音识别<br/>

# 一、简介
本项目可以在[AI Studio](https://aistudio.baidu.com/aistudio/projectdetail/1597936)在线运行。

PPASR基于PaddlePaddle实现的端到端自动语音识别，本项目最大的特点简单，在保证准确率不低的情况下，项目尽量做得浅显易懂，能够让每个想入门语音识别的开发者都能够轻松上手。PPASR只使用卷积神经网络，无其他特殊网络结构，模型简单易懂，且是端到端的，不需要音频对齐，因为本项目使用了CTC Loss作为损失函数。在传统的语音识别的模型中，我们对语音模型进行训练之前，往往都要将文本与语音进行严格的对齐操作。在传统的语音识别的模型中，我们对语音模型进行训练之前，往往都要将文本与语音进行严格的对齐操作，这种对齐非常浪费时间，而且对齐之后，模型预测出的label只是局部分类的结果，而无法给出整个序列的输出结果，往往要对预测出的label做一些后处理才可以得到我们最终想要的结果。基于这种情况，就出现了CTC（Connectionist temporal classification），使用CTC Loss就不需要进行音频对齐，直接输入是一句完整的语音数据，输出的是整个序列结果，这种情况OCR也是同样的情况。

# 二、安装环境

 - 本项目可以在Windows或者Ubuntu都可以运行，安装环境很简单，只需要执行以下命令即可。

In [ ]:
!pip install numpy==1.19.3 scipy==1.6.1 tqdm pytest-runner librosa==0.6.3 python-Levenshtein==0.12.2 visualdl==2.1.1 --user
!pip install SoundFile==0.9.0.post1 --user

# 三、数据准备

1. 在`data`目录下是公开数据集的下载和制作训练数据列表和字典的，本项目提供了下载公开的中文普通话语音数据集，分别是Aishell，Free ST-Chinese-Mandarin-Corpus，THCHS-30 这三个数据集，总大小超过28G。下载这三个数据只需要执行一下代码即可，当然如果想快速训练，也可以只下载其中一个。特别提醒，这样下载会比较慢，最好是自己上传数据集。


In [ ]:
# !python3 data1/aishell.py
#!python3 data1/free_st_chinese_mandarin_corpus.py
!python3 data1/thchs_30.py

 - 如果开发者有自己的数据集，可以使用自己的数据集进行训练，当然也可以跟上面下载的数据集一起训练。自定义的语音数据需要符合一下格式：
    1. 语音文件需要放在`dataset/audio/`目录下，例如我们有个`wav`的文件夹，里面都是语音文件，我们就把这个文件存放在`dataset/audio/`。
    2. 然后把数据列表文件存在`dataset/annotation/`目录下，程序会遍历这个文件下的所有数据列表文件。例如这个文件下存放一个`my_audio.txt`，它的内容格式如下。每一行数据包含该语音文件的相对路径和该语音文件对应的中文文本，要注意的是该中文文本只能包含纯中文，不能包含标点符号、阿拉伯数字以及英文字母。
```shell script
dataset/audio/wav/0175/H0175A0171.wav 我需要把空调温度调到二十度
dataset/audio/wav/0175/H0175A0377.wav 出彩中国人
dataset/audio/wav/0175/H0175A0470.wav 据克而瑞研究中心监测
dataset/audio/wav/0175/H0175A0180.wav 把温度加大到十八

 - 执行下面的命令，创建数据列表，以及建立词表，也就是数据字典，把所有出现的字符都存放子在`zh_vocab.json`文件中，生成的文件都存放在`dataset/`目录下。最最最重要的是还计算了数据集的均值和标准值，计算得到的均值和标准值需要更新在训练参数`data_mean`和`data_std`中，之后的评估和预测同样需要用到。有几个参数需要注意，参数`is_change_frame_rate`是指定在生成数据集的时候，是否要把音频的采样率转换为16000Hz，最好是使用默认值。参数`min_duration`和`max_duration`限制音频的长度，特别是有些音频太长，会导致显存不足，训练直接崩掉。

我们来说说这些文件和数据的具体作用，创建数据列表是为了在训练是读取数据，读取数据程序通过读取图像列表的每一行都能得到音频的文件路径、音频长度以及这句话的内容。通过路径读取音频文件并进行预处理，音频长度用于统计数据总长度，文字内容就是输入数据的标签，在训练是还需要数据字典把这些文字内容转置整型的数字，比如`是`这个字在数据字典中排在第5，那么它的标签就是4，标签从0开始。至于最后生成的均值和标准值，因为我们的数据在训练之前还需要归一化，因为每个数据的分布不一样，不同图像，最大最小值都是确定的，所以我们要统计一批数据来计算均值和标准值，之后的数据的归一化都使用这个均值和标准值。


In [ ]:
!python3 create_manifest.py

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
-----------  Configuration Arguments -----------
annotation_path: dataset/annotation/
count_threshold: 0
is_change_frame_rate: True
manifest_path: dataset/manifest.train
manifest_prefix: dataset/
vocab_path: dataset/zh_vocab.json
------------------------------------------------
开始生成数据列表...
100%|███████████████████████████████████| 13388/13388 [00:06<00:00, 2131.76it/s]
完成生成数据列表，数据集总长度为34.16小时！
开始生成数据字典...
100%|██████████████████████████████████| 13254/13254 [00:00<00:00, 17726.18it/s]
数据字典生成完成！
开始抽取10%的数据计算均值和标准值...
100%|█████████████████████████████████████| 13254/13254 [15:32<00:00, 14.21it/s]
【特别重要】：均值：-2.427870, 标准值：44.181725, 请根据这两个值修改训练参数！


可以用使用`python create_manifest.py --help`命令查看各个参数的说明和默认值。
```shell
usage: create_manifest.py [-h] [----annotation_path ANNOTATION_PATH]
                          [--manifest_prefix MANIFEST_PREFIX]
                          [--is_change_frame_rate IS_CHANGE_FRAME_RATE]
                          [--min_duration MIN_DURATION]
                          [--max_duration MAX_DURATION]
                          [--count_threshold COUNT_THRESHOLD]
                          [--vocab_path VOCAB_PATH]
                          [--manifest_path MANIFEST_PATH]

optional arguments:
  -h, --help            show this help message and exit
  ----annotation_path ANNOTATION_PATH
                        标注文件的路径 默认: dataset/annotation/.
  --manifest_prefix MANIFEST_PREFIX
                        训练数据清单，包括音频路径和标注信息 默认: dataset/.
  --is_change_frame_rate IS_CHANGE_FRAME_RATE
                        是否统一改变音频为16000Hz，这会消耗大量的时间 默认: True.
  --min_duration MIN_DURATION
                        过滤最短的音频长度 默认: 0.
  --max_duration MAX_DURATION
                        过滤最长的音频长度，当为-1的时候不限制长度 默认: 20.
  --count_threshold COUNT_THRESHOLD
                        字符计数的截断阈值，0为不做限制 默认: 0.
  --vocab_path VOCAB_PATH
                        生成的数据字典文件 默认: dataset/zh_vocab.json.
  --manifest_path MANIFEST_PATH
                        数据列表路径 默认: dataset/manifest.train.
```

# 四、训练模型
创建数据列表之后就可以就可以开始训练语音识别模型了。

## 4.1 创建模型
PPASR模型是一个只使用卷积层的模型，并没有使用更加复杂的RNN模型，以下就是使用PaddlePaddle实现的一个语音识别模型。使用动态图自定义网络模型非常简单。

In [ ]:
%%writefile utils/model.py
import paddle
import paddle.nn as nn
from paddle.nn.initializer import KaimingNormal


# 门控线性单元 Gated Linear Units (GLU)
class GLU(nn.Layer):
    def __init__(self, axis):
        super(GLU, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.axis = axis

    def forward(self, x):
        a, b = paddle.split(x, num_or_sections=2, axis=self.axis)
        act_b = self.sigmoid(b)
        out = paddle.multiply(x=a, y=act_b)
        return out


# 基本卷积块
class ConvBlock(nn.Layer):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, p=0.5):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1D(in_channels, out_channels, kernel_size, stride, padding, weight_attr=KaimingNormal())
        self.conv = nn.utils.weight_norm(self.conv)
        self.act = GLU(axis=1)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = self.conv(x)
        x = self.act(x)
        x = self.dropout(x)
        return x


# PPASR模型
class PPASR(nn.Layer):
    def __init__(self, vocabulary, data_mean=None, data_std=None, name="PPASR"):
        super(PPASR, self).__init__(name_scope=name)
        # 数据均值和标准值到模型中，方便以后推理使用
        if data_mean is None:
            data_mean = paddle.to_tensor(1.0)
        if data_std is None:
            data_std = paddle.to_tensor(1.0)
        self.register_buffer("data_mean", data_mean, persistable=True)
        self.register_buffer("data_std", data_std, persistable=True)
        # 模型的输出大小，字典大小+1
        self.output_units = len(vocabulary) + 1
        self.conv1 = ConvBlock(128, 500, 48, 2, padding=97, p=0.2)
        self.conv2 = ConvBlock(250, 500, 7, 1, p=0.3)
        self.conv3 = ConvBlock(250, 2000, 32, 1, p=0.3)
        self.conv4 = ConvBlock(1000, 2000, 1, 1, p=0.3)
        self.out = nn.utils.weight_norm(nn.Conv1D(1000, self.output_units, 1, 1))

    def forward(self, x, input_lens=None):
        x = self.conv1(x)
        for i in range(7):
            x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.out(x)
        if input_lens is not None:
            return x, paddle.to_tensor(input_lens / 2 + 1, dtype='int64')
        return x


Overwriting utils/model.py


## 4.2 解码方法

模型输出的结果需要解码才能得到真实的文本结果，本项目使用的是贪心策略解码方法，贪心策略是在每一步选择概率最大的输出值，这样就可以得到最终解码的输出序列。然而，CTC网络的输出序列只对应了搜索空间的一条路径，一个最终标签可对应搜索空间的N条路径，所以概率最大的路径并不等于最终标签的概率最大，即不是最优解。

In [ ]:
%%writefile utils/decoder.py
import Levenshtein as Lev
import paddle


class GreedyDecoder(object):
    def __init__(self, vocabulary, blank_index=0):
        self.int_to_char = dict([(i, c) for (i, c) in enumerate(vocabulary)])
        self.blank_index = blank_index

    # 给定一个数字序列列表，返回相应的字符串
    def convert_to_strings(self, sequences, sizes=None, remove_repetitions=False, return_offsets=False):
        strings = []
        offsets = [] if return_offsets else None
        for x in range(len(sequences)):
            seq_len = sizes[x] if sizes is not None else len(sequences[x])
            string, string_offsets = self.process_string(sequences[x], seq_len, remove_repetitions)
            strings.append([string])
            if return_offsets:
                offsets.append([string_offsets])
        if return_offsets:
            return strings, offsets
        else:
            return strings

    # 获取字符，并删除重复的字符
    def process_string(self, sequence, size, remove_repetitions=False):
        string = ""
        offsets = []
        sequence = sequence.numpy()
        for i in range(size):
            char = self.int_to_char[sequence[i].item()]
            if char != self.int_to_char[self.blank_index]:
                # 是否删除重复的字符
                if remove_repetitions and i != 0 and char == self.int_to_char[sequence[i - 1].item()]:
                    pass
                else:
                    string = string + char
                    offsets.append(i)
        return string, paddle.to_tensor(offsets, dtype='int64')

    def cer(self, s1, s2):
        """
       通过计算两个字符串的距离，得出字错率
        """
        s1, s2, = s1.replace(" ", ""), s2.replace(" ", "")
        return Lev.distance(s1, s2)

    def decode(self, probs, sizes=None):
        """
        解码，传入结果的概率解码得到字符串，删除序列中的重复元素和空格。
        """
        max_probs = paddle.argmax(probs, 2)
        strings, offsets = self.convert_to_strings(
            max_probs,
            sizes,
            remove_repetitions=True,
            return_offsets=True)
        return strings, offsets


Overwriting utils/decoder.py


## 4.3 数据读取
在读取音频数据的同时也在做预处理，本项目主要是将音频执行梅尔频率倒谱系数(MFCCs)处理，然后在使用出来的数据进行训练，在读取音频时，使用`librosa.load(wav_path, sr=16000)`函数读取音频文件，再使用`librosa.feature.mfcc()`执行数据处理。MFCC全称梅尔频率倒谱系数。梅尔频率是基于人耳听觉特性提出来的， 它与Hz频率成非线性对应关系。梅尔频率倒谱系数(MFCC)则是利用它们之间的这种关系，计算得到的Hz频谱特征，主要计算方式分别是预加重，分帧，加窗，快速傅里叶变换(FFT)，梅尔滤波器组，离散余弦变换(DCT)，最后提取语音数据特征和降低运算维度。本项目使用的全部音频的采样率都是16000Hz，如果其他采样率的音频都需要转为16000Hz，`create_manifest.py`程序也提供了把音频转为16000Hz。


In [ ]:
%%writefile utils/data.py
import json
import wave

import librosa
import numpy as np
import soundfile
from paddle.io import Dataset


# 加载二进制音频文件，转成短时傅里叶变换
def load_audio_stft(wav_path, mean=None, std=None):
    with wave.open(wav_path) as wav:
        wav = np.frombuffer(wav.readframes(wav.getnframes()), dtype="int16").astype("float32")
    stft = librosa.stft(wav, n_fft=255, hop_length=160, win_length=200, window="hamming")
    spec, phase = librosa.magphase(stft)
    spec = np.log1p(spec)
    if mean is not None and std is not None:
        spec = (spec - mean) / std
    return spec


# 读取音频文件转成梅尔频率倒谱系数(MFCCs)
def load_audio_mfcc(wav_path, mean=None, std=None):
    wav, sr = librosa.load(wav_path, sr=16000)
    mfccs = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=128, n_fft=512, hop_length=128).astype("float32")
    if mean is not None and std is not None:
        mfccs = (mfccs - mean) / std
    return mfccs


# 改变音频采样率为16000Hz
def change_rate(audio_path):
    data, sr = soundfile.read(audio_path)
    if sr != 16000:
        data = librosa.resample(data, sr, target_sr=16000)
        soundfile.write(audio_path, data, samplerate=16000)


# 音频数据加载器
class PPASRDataset(Dataset):
    def __init__(self, data_list, dict_path, mean=None, std=None, min_duration=0, max_duration=-1):
        super(PPASRDataset, self).__init__()
        self.mean = mean
        self.std = std
        # 获取数据列表
        with open(data_list, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        self.data_list = []
        for line in lines:
            line = json.loads(line)
            # 跳过超出长度限制的音频
            if line["duration"] < min_duration:
                continue
            if max_duration != -1 and line["duration"] > max_duration:
                continue
            self.data_list.append([line["audio_path"], line["text"]])
        # 加载数据字典
        with open(dict_path, 'r', encoding='utf-8') as f:
            labels = eval(f.read())
        self.vocabulary = dict([(labels[i], i) for i in range(len(labels))])

    def __getitem__(self, idx):
        # 分割音频路径和标签
        wav_path, transcript = self.data_list[idx]
        # 读取音频并转换为梅尔频率倒谱系数(MFCCs)
        mfccs = load_audio_mfcc(wav_path, self.mean, self.std)
        # 将字符标签转换为int数据
        transcript = list(filter(None, [self.vocabulary.get(x) for x in transcript]))
        transcript = np.array(transcript, dtype='int32')
        return mfccs, transcript

    def __len__(self):
        return len(self.data_list)


# 对一个batch的数据处理
def collate_fn(batch):
    # 找出音频长度最长的
    batch = sorted(batch, key=lambda sample: sample[0].shape[1], reverse=True)
    freq_size = batch[0][0].shape[0]
    max_audio_length = batch[0][0].shape[1]
    batch_size = len(batch)
    # 找出标签最长的
    batch_temp = sorted(batch, key=lambda sample: len(sample[1]), reverse=True)
    max_label_length = len(batch_temp[0][1])
    # 以最大的长度创建0张量
    inputs = np.zeros((batch_size, freq_size, max_audio_length), dtype='float32')
    labels = np.zeros((batch_size, max_label_length), dtype='int32')
    input_lens = []
    label_lens = []
    for x in range(batch_size):
        sample = batch[x]
        tensor = sample[0]
        target = sample[1]
        seq_length = tensor.shape[1]
        label_length = target.shape[0]
        # 将数据插入都0张量中，实现了padding
        inputs[x, :, :seq_length] = tensor[:, :]
        labels[x, :label_length] = target[:]
        input_lens.append(seq_length)
        label_lens.append(len(target))
    input_lens = np.array(input_lens, dtype='int64')
    label_lens = np.array(label_lens, dtype='int64')
    return inputs, labels, input_lens, label_lens


Overwriting utils/data.py


## 4.4 开始训练
开始训练语音识别模型， 每训练一轮保存一次模型，模型保存在`models/`目录下，测试使用的是贪心解码路径解码方法。本项目支持多卡训练，在没有指定`CUDA_VISIBLE_DEVICES`时，会使用全部的GPU进行执行训练，也可以指定某几个GPU训练，如`export CUDA_VISIBLE_DEVICES=0,1`指定使用第1张和第2张显卡训练。除了参数`data_mean`和`data_std`需要根据计算的结果修改，其他的参数一般不需要改动，参数`num_workers`可以更加CPU的核数修改，这个参数是指定使用多少个线程读取数据。参数`pretrained_model`是指定预训练模型所在的文件夹，如果使用训练模型，必须使用跟预训练配套的数据字典，原因是，其一，数据字典的大小指定了模型的输出大小，如果使用了其他更大的数据字典，预训练模型就无法完全加载。其二，数值字典定义了文字的ID，不同的数据字典文字的ID可能不一样，这样预训练模型的作用就不是那么大了。

In [5]:
import argparse
import functools
import os
from datetime import datetime

import numpy as np
import paddle
import paddle.distributed as dist
from paddle.io import DataLoader
from visualdl import LogWriter

from utils.data import PPASRDataset, collate_fn
from utils.decoder import GreedyDecoder
from utils.model import PPASR


# 训练的批量大小
batch_size = 32
# 读取数据的线程数量
num_workers = 4
# 训练的轮数
num_epoch = 100
# 初始学习率的大小
learning_rate = 1e-3
# 数据集的均值
data_mean = -2.427870
# 数据集的标准值
data_std = 44.181725
# 过滤最短的音频长度
min_duration = 0
# 过滤最短的音频长度
max_duration = 20
# 训练数据的数据列表路径
train_manifest = 'dataset/manifest.train'
# 测试数据的数据列表路径
test_manifest = 'dataset/manifest.test'
# 数据字典的路径
dataset_vocab = 'dataset/zh_vocab.json'
# 模型保存的路径
save_model = 'models/'
# 预训练模型的路径，当为None则不使用预训练模型
pretrained_model = None


# 评估模型
def evaluate(model, test_loader, greedy_decoder):
    cer = []
    for batch_id, (inputs, labels, _, _) in enumerate(test_loader()):
        # 执行识别
        outs = model(inputs)
        outs = paddle.nn.functional.softmax(outs, 1)
        outs = paddle.transpose(outs, perm=[0, 2, 1])
        # 解码获取识别结果
        out_strings, out_offsets = greedy_decoder.decode(outs)
        labels = greedy_decoder.convert_to_strings(labels)
        for out_string, label in zip(*(out_strings, labels)):
            # 计算字错率
            c = greedy_decoder.cer(out_string[0], label[0]) / float(len(label[0]))
            cer.append(c)
    cer = float(np.mean(cer))
    return cer


# 保存模型
def save_model(epoch, model, optimizer):
    model_path = os.path.join(save_model, 'epoch_%d' % epoch)
    if epoch == num_epoch - 1:
        model_path = os.path.join(save_model, 'step_final')
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    paddle.save(model.state_dict(), os.path.join(model_path, 'model.pdparams'))
    paddle.save(optimizer.state_dict(), os.path.join(model_path, 'optimizer.pdopt'))


def train():
    # 日志记录器
    writer = LogWriter(logdir='log')
    # 设置支持多卡训练
    dist.init_parallel_env()
    # 获取训练数据
    train_dataset = PPASRDataset(train_manifest, dataset_vocab,
                                 mean=data_mean,
                                 std=data_std,
                                 min_duration=min_duration,
                                 max_duration=max_duration)
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=batch_size,
                              collate_fn=collate_fn,
                              num_workers=num_workers,
                              use_shared_memory=False)
    train_loader_shuffle = DataLoader(dataset=train_dataset,
                                      batch_size=batch_size,
                                      collate_fn=collate_fn,
                                      num_workers=num_workers,
                                      shuffle=True,
                                      use_shared_memory=False)
    # 获取测试数据
    test_dataset = PPASRDataset(test_manifest, dataset_vocab, mean=data_mean, std=data_std)
    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             collate_fn=collate_fn,
                             num_workers=num_workers,
                             use_shared_memory=False)
    # 获取解码器，用于评估
    greedy_decoder = GreedyDecoder(train_dataset.vocabulary)
    # 获取模型，同时数据均值和标准值到模型中，方便以后推理使用
    model = PPASR(train_dataset.vocabulary, data_mean=paddle.to_tensor(data_mean), data_std=paddle.to_tensor(data_std))
    print('input_size的第三个参数是变长的，这里为了能查看输出的大小变化，指定了一个值！')
    paddle.summary(model, input_size=(batch_size, 128, 500))
    # 设置支持多卡训练
    model = paddle.DataParallel(model)
    # 设置优化方法
    clip = paddle.nn.ClipGradByNorm(clip_norm=1.0)
    boundaries = [10, 20, 50]
    lr = [0.1 ** l * learning_rate for l in range(len(boundaries) + 1)]
    scheduler = paddle.optimizer.lr.PiecewiseDecay(boundaries=boundaries, values=lr, verbose=True)
    optimizer = paddle.optimizer.Adam(parameters=model.parameters(),
                                      learning_rate=scheduler,
                                      grad_clip=clip)
    # 获取损失函数
    ctc_loss = paddle.nn.CTCLoss()
    # 加载预训练模型
    if pretrained_model is not None:
        model.set_state_dict(paddle.load(os.path.join(pretrained_model, 'model.pdparams')))
        optimizer.set_state_dict(paddle.load(os.path.join(pretrained_model, 'optimizer.pdopt')))
    train_step = 0
    test_step = 0
    # 开始训练
    for epoch in range(num_epoch):
        # 第一个epoch不打乱数据
        if epoch == 1:
            train_loader = train_loader_shuffle
        for batch_id, (inputs, labels, input_lens, label_lens) in enumerate(train_loader()):
            out, out_lens = model(inputs, input_lens)
            out = paddle.transpose(out, perm=[2, 0, 1])
            # 计算损失
            loss = ctc_loss(out, labels, out_lens, label_lens)
            loss.backward()
            optimizer.step()
            optimizer.clear_grad()
            # 多卡训练只使用一个进程打印
            if batch_id % 100 == 0 and dist.get_rank() == 0:
                print('[%s] Train epoch %d, batch %d, loss: %f' % (datetime.now(), epoch, batch_id, loss))
                writer.add_scalar('Train loss', loss, train_step)
                train_step += 1
            # 固定步数也要保存一次模型
            if batch_id % 2000 == 0 and batch_id != 0:
                # 保存模型
                save_model(epoch=epoch, model=model, optimizer=optimizer)
        # 执行评估
        model.eval()
        cer = evaluate(model, test_loader, greedy_decoder)
        print('[%s] Test epoch %d, cer: %f' % (datetime.now(), epoch, cer))
        writer.add_scalar('Test cer', cer, test_step)
        test_step += 1
        model.train()
        # 记录学习率
        writer.add_scalar('Learning rate', scheduler.last_lr, epoch)
        # 保存模型
        save_model(epoch=epoch, model=model, optimizer=optimizer)
        scheduler.step()


if __name__ == '__main__':
    train()

input_size的第三个参数是变长的，这里为了能查看输出的大小变化，指定了一个值！
---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv1D-21      [[32, 128, 500]]      [32, 500, 324]       3,073,000   
  Sigmoid-17      [[32, 250, 324]]      [32, 250, 324]           0       
    GLU-17        [[32, 500, 324]]      [32, 250, 324]           0       
  Dropout-17      [[32, 250, 324]]      [32, 250, 324]           0       
 ConvBlock-17     [[32, 128, 500]]      [32, 250, 324]           0       
   Conv1D-22      [[32, 250, 288]]      [32, 500, 282]        876,000    
  Sigmoid-18      [[32, 250, 282]]      [32, 250, 282]           0       
    GLU-18        [[32, 500, 282]]      [32, 250, 282]           0       
  Dropout-18      [[32, 250, 282]]      [32, 250, 282]           0       
 ConvBlock-18     [[32, 250, 288]]      [32, 250, 282]           0       
   Conv1D-23      [[32, 250, 282]]     [32, 2000, 251]      16,004

[2021-05-12 10:47:56.302111] Train epoch 0, batch 0, loss: 168.749115
[2021-05-12 11:02:46.394364] Train epoch 0, batch 100, loss: 7.148998


在训练过程中会保存VisualDL日志到log文件夹中，可以通过VisualDL可视化功能查看

![](https://ai-studio-static-online.cdn.bcebos.com/c7516b345d0843c1befb52fada3ed2e4f7c707e537604ca88b803f5c7ac87672)



# 五、评估和预测

在评估和预测中，对结果解码的贪心策略解码方法，贪心策略是在每一步选择概率最大的输出值，这样就可以得到最终解码的输出序列。然而，CTC网络的输出序列只对应了搜索空间的一条路径，一个最终标签可对应搜索空间的N条路径，所以概率最大的路径并不等于最终标签的概率最大，即不是最优解。但贪心策略是最简单易懂且快速地一种方法。在语音识别上使用最多的解码方法还有定向搜索策略，这种策略准确率更高，同时也相对复杂，解码速度也相对慢很多。

## 5.1 评估
我们可以使用这个脚本对模型进行评估，通过字符错误率来评价模型的性能。目前只支持贪心策略解码方法。在评估中音频预处理的`mean`和`std`需要跟训练时一样，但这里不需要开发者手动指定，因为这两个参数在训练的时候就已经保持在模型中，这时只需从模型中读取这两个参数的值就可以。参数`model_path`指定模型所在的文件夹的路径。


In [ ]:
import argparse
import functools
import os
import time

import numpy as np
import paddle
from paddle.io import DataLoader
from tqdm import tqdm
from utils.data import PPASRDataset, collate_fn
from utils.decoder import GreedyDecoder
from utils.model import PPASR

# 训练的批量大小
batch_size = 32
# 读取数据的线程数量
num_workers = 8
# 测试数据的数据列表路径
test_manifest = 'dataset/manifest.test'
# 数据字典的路径
dataset_vocab = 'dataset/zh_vocab.json'
# 模型的路径
model_path = 'models/step_final/'

# 获取测试数据
test_dataset = PPASRDataset(test_manifest, dataset_vocab)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         collate_fn=collate_fn,
                         num_workers=num_workers,
                         use_shared_memory=False)
# 获取解码器，用于评估
greedy_decoder = GreedyDecoder(test_dataset.vocabulary)
# 获取模型
model = PPASR(test_dataset.vocabulary)
model.set_state_dict(paddle.load(os.path.join(model_path, 'model.pdparams')))
# 获取保存在模型中的数据均值和标准值，设置数据处理器
test_dataset.mean = model.data_mean.numpy()[0]
test_dataset.std = model.data_std.numpy()[0]
model.eval()


# 评估模型
def evaluate():
    cer = []
    for batch_id, (inputs, labels, _, _) in enumerate(tqdm(test_loader())):
        # 执行识别
        outs = model(inputs)
        outs = paddle.nn.functional.softmax(outs, 1)
        outs = paddle.transpose(outs, perm=[0, 2, 1])
        # 解码获取识别结果
        out_strings, out_offsets = greedy_decoder.decode(outs)
        labels = greedy_decoder.convert_to_strings(labels)
        for out_string, label in zip(*(out_strings, labels)):
            # 计算字错率
            c = greedy_decoder.cer(out_string[0], label[0]) / float(len(label[0]))
            cer.append(c)
    cer = float(np.mean(cer))
    return cer

start = time.time()
cer = evaluate()
end = time.time()
print('识别时间：%dms，字错率：%f' % (round((end - start) * 1000), cer))

100%|██████████| 5/5 [00:34<00:00,  6.89s/it]

识别时间：35071ms，字错率：0.025435


## 5.2 预测
我们可以使用这个脚本使用模型进行预测，通过传递音频文件的路径进行识别。在预测中音频预处理的`mean`和`std`需要跟训练时一样，但这里不需要开发者手动指定，因为这两个参数在训练的时候就已经保持在模型中，这时只需从模型中读取这两个参数的值就可以。参数`model_path`指定模型所在的文件夹的路径，参数`wav_path`指定需要预测音频文件的路径。


In [ ]:
import functools
import os
import time

import paddle

from utils.data import load_audio_mfcc
from utils.decoder import GreedyDecoder
from utils.model import PPASR


# 用于识别的音频路径
audio_path = 'dataset/test.wav'
# 数据字典的路径
dataset_vocab = 'dataset/zh_vocab.json'
# 模型的路径
model_path = 'models/step_final/'

# 加载数据字典
with open(dataset_vocab, 'r', encoding='utf-8') as f:
    labels = eval(f.read())
vocabulary = dict([(labels[i], i) for i in range(len(labels))])
# 获取解码器
greedy_decoder = GreedyDecoder(vocabulary)

# 创建模型
model = PPASR(vocabulary)
model.set_state_dict(paddle.load(os.path.join(model_path, 'model.pdparams')))
# 获取保存在模型中的数据均值和标准值
data_mean = model.data_mean.numpy()[0]
data_std = model.data_std.numpy()[0]
model.eval()


def infer():
    # 读取音频文件转成梅尔频率倒谱系数(MFCCs)
    mfccs = load_audio_mfcc(audio_path, mean=data_mean, std=data_std)

    mfccs = paddle.to_tensor(mfccs, dtype='float32')
    mfccs = paddle.unsqueeze(mfccs, axis=0)
    # 执行识别
    out = model(mfccs)
    out = paddle.nn.functional.softmax(out, 1)
    out = paddle.transpose(out, perm=[0, 2, 1])
    # 执行解码
    out_string, out_offset = greedy_decoder.decode(out)
    return out_string[0][0]


start = time.time()
result_text = infer()
end = time.time()
print('识别时间：%dms，识别结果：%s' % (round((end - start) * 1000), result_text))

识别时间：795ms，识别结果：柳宗夏现年六十岁五十年代基入韩外交部工作一九九四年十二月任外交安保首席秘书


# 六、参考资料

1. https://github.com/yeyupiaoling/MASR
2. https://www.paddlepaddle.org.cn